### Extração Manual

In [4]:
%%capture
%pip install pandas matplotlib requests

In [5]:
import zipfile, requests, os
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
FLOAT_COLS = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]

CATEGORY_COLS = [
    'uf',
    'tcb',
    'sr',
    'cliente'
    'ocupacao',
    'cnae_secao',
    'cnae_subclasse',
    'cliente', 
    'ocupacao',
    'porte',
    'modalidade',
    'origem',
    'indexador'
]

INT_COLS = [
    'numero_de_operacoes'
]

def convert_category_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].astype('category')

    return df

def convert_float_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace(",","."))
    df[column] = df[column].astype('float64')

    return df

def convert_int_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace("<= 15","15"))
    df[column] = df[column].astype(int)

    return df

In [7]:
planilhas = [
	# 'planilha_2012',
	# 'planilha_2013',
	# 'planilha_2014',
	# 'planilha_2015',
	# 'planilha_2016',
	# 'planilha_2017',
	# 'planilha_2018',
	# 'planilha_2019',
	# 'planilha_2020',
	# 'planilha_2021',
	# 'planilha_2022',
	# 'planilha_2023',
	'planilha_2024'
]

for planilha in planilhas:
	if not os.path.exists(f'data/{planilha}.zip'):
		url = f'https://www.bcb.gov.br/pda/desig/{planilha}.zip'

		response = requests.get(url)

		with open(f'data/{planilha}.zip', 'wb') as f:
			f.write(response.content)

In [8]:
for planilha in planilhas:
	zip_obj = zipfile.ZipFile(f'data/{planilha}.zip', 'r')
	zip_obj.extractall(f'data/planilha_2024')
	print(f'Extraído {planilha}')

Extraído planilha_2024


In [9]:
files = os.listdir('data/planilha_2024')
files.sort()
df = pd.read_csv(f'data/planilha_2024/{files[-1]}', sep=';', encoding='utf-8')

In [10]:
# files = os.listdir('data/planilhas')
# files.sort()
# merged_df = pd.DataFrame()
# for file in files:
# 	df = pd.read_csv(f'data/planilhas/{file}', sep=';', encoding='utf-8')
# 	merged_df = pd.concat([merged_df, df])
# 	print(f'Processado o arquivo {file}, com {len(df.columns)} colunas')

In [11]:
# merged_df.to_parquet('data/merged.parquet')

### Tratamento

## AJUSTE TIPAGEM DOS DADOS

In [12]:
CATEGORY_COLS

['uf',
 'tcb',
 'sr',
 'clienteocupacao',
 'cnae_secao',
 'cnae_subclasse',
 'cliente',
 'ocupacao',
 'porte',
 'modalidade',
 'origem',
 'indexador']

In [13]:
df['uf']

0         AC
1         AC
2         AC
3         AC
4         AC
          ..
925594    TO
925595    TO
925596    TO
925597    TO
925598    TO
Name: uf, Length: 925599, dtype: object

In [14]:
df['data_base'] = pd.to_datetime(df['data_base'], format="%Y-%m-%d")

for column in df.columns:

	if column in FLOAT_COLS:
		df = convert_float_dtype(df=df, column=column)
		df = df.rename({column: "vl_" + column.lower()}, regex=True)

	if column in CATEGORY_COLS:
		df = convert_category_dtype(df=df, column=column)
		df = df.rename({column: "ct_" + column.lower()}, regex=True)

	if column in INT_COLS:
		df = convert_int_dtype(df=df, column=column)
		df = df.rename({column: "nu_" + column.lower()}, regex=True)

TypeError: rename() got an unexpected keyword argument 'regex'

In [10]:
df[['cliente', 'ocupacao']]

,cliente,ocupacao
0,PF,PF - Aposentado/pensionista
1,PF,PF - Aposentado/pensionista
2,PF,PF - Aposentado/pensionista
3,PF,PF - Aposentado/pensionista
4,PF,PF - Aposentado/pensionista
...,...,...
925594,PJ,-
925595,PJ,-
925596,PJ,-
925597,PJ,-


In [11]:
df.columns

Index(['data_base', 'uf', 'tcb', 'sr', 'cliente', 'ocupacao', 'cnae_secao',
       'cnae_subclasse', 'porte', 'modalidade', 'origem', 'indexador',
       'numero_de_operacoes', 'a_vencer_ate_90_dias',
       'a_vencer_de_91_ate_360_dias', 'a_vencer_de_361_ate_1080_dias',
       'a_vencer_de_1081_ate_1800_dias', 'a_vencer_de_1801_ate_5400_dias',
       'a_vencer_acima_de_5400_dias', 'vencido_acima_de_15_dias',
       'carteira_ativa', 'carteira_inadimplida_arrastada',
       'ativo_problematico'],
      dtype='object')

In [18]:
df['data_base'].value_counts()

data_base
2024-07-31    925599
Name: count, dtype: int64

In [13]:
df['numero_de_operacoes'].value_counts()

numero_de_operacoes
15        698300
16          7209
17          6511
18          6022
19          5654
           ...  
19839          1
65897          1
11533          1
112712         1
31291          1
Name: count, Length: 13769, dtype: int64

In [14]:
df_menor_15['a_vencer_ate_90_dias'].value_counts()

NameError: name 'df_menor_15' is not defined

In [71]:
df

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,2024-07-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,...,3573737.49,1074965.55,112570.63,8832.10,12332.61,0.00,189197.15,4971635.53,129021.18,235613.15
1,2024-07-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,...,1267084.96,4188864.71,9724275.32,7787093.59,8248954.43,0.00,190861.03,31407134.04,1228674.44,1297992.21
2,2024-07-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,...,0.00,0.00,0.00,0.00,0.00,0.00,45090.47,45090.47,45090.47,45090.47
3,2024-07-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,...,242763.31,886217.49,1255335.50,546989.13,242992.67,0.00,24701.58,3198999.68,36282.35,85970.39
4,2024-07-31,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Habitacional,...,89275.67,248196.04,492695.37,285504.96,579527.45,75215.48,0.00,1770414.97,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925594,2024-07-31,TO,Não bancário,S5,PJ,-,"PJ - Transporte, armazenagem e correio","PJ - Transporte rodoviário de carga, exceto pr...",PJ - Médio,PJ - Investimento,...,0.00,345785.14,523748.00,0.00,0.00,0.00,0.00,869533.14,0.00,0.00
925595,2024-07-31,TO,Não bancário,S5,PJ,-,"PJ - Transporte, armazenagem e correio","PJ - Transporte rodoviário de carga, exceto pr...",PJ - Pequeno,PJ - Outros créditos,...,248.05,109.34,0.00,0.00,0.00,0.00,547.84,905.23,354.33,905.23
925596,2024-07-31,TO,Não bancário,S5,PJ,-,"PJ - Transporte, armazenagem e correio","PJ - Transporte rodoviário de carga, exceto pr...",PJ - Micro,PJ - Outros créditos,...,6732.11,523.27,0.00,0.00,0.00,0.00,0.00,7255.38,0.00,0.00
925597,2024-07-31,TO,Não bancário,S5,PJ,-,"PJ - Transporte, armazenagem e correio","PJ - Transporte rodoviário de carga, exceto pr...",PJ - Médio,PJ - Outros créditos,...,37638.07,0.00,0.00,0.00,0.00,0.00,0.00,37638.07,0.00,0.00


In [12]:
df_limpo = df.dropna()

In [28]:
df['numero_de_operacoes'].copy()

numero_de_operacoes
<= 15     698300
16          7209
17          6511
18          6022
19          5654
           ...  
19839          1
65897          1
11533          1
112712         1
31291          1
Name: count, Length: 13769, dtype: int64